In [2]:
# Importo le librerie
import requests
import pandas as pd
from serpapi import GoogleScholarSearch
import re
import random
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from IPython.display import display
import pickle
from pyvis.network import Network
import json

In [8]:
# Imposto Key
GoogleScholarSearch.SERP_API_KEY = "5137659bcfa1c48c5ef9cb22d32468b7c6f58739e298365157e18a98ecdbb832"

In [4]:
# 1. A partire da nodes.csv, utilizzare la libreria Python SerpAPI per scaricare, per ciascuno dei 7 autori elencati:
# - author_id: ID identificativo del profilo Google Scholar
# - cited_by: numero totale di citazioni ricevute
# - interests: elenco degli interessi di ciascun autore
# Sfruttando la libreria Python Pandas, usare la struttura dati DataFrame per
# aggiornare il file originale con apposite colonne e memorizzarlo nella cartella /data.

# Leggo file csv e carico dataframe
df_nodes = pd.read_csv("data/nodes.csv")
# display(df_nodes)
    
# Creo dataframe ausiliario
df_nodes_aux = pd.DataFrame(columns=["author_id", "cited_by", "interests"])

# Itero per la coppia name e affilations
for name, affilations in zip(df_nodes["name"], df_nodes["affiliations"]):

    # Ricerco name tra i profili presenti su Google Scholar
    params = {
        "engine": "google_scholar_profiles",
        "hl": "en",
        "mauthors": name
    }

    search = GoogleScholarSearch(params)
    results = search.get_dict()["profiles"]

    # Tramite affilations so quale profilo scaricare
    correct_affilations = None

    for result in results:
        if result["affiliations"] == affilations:
            correct_affilations = result

    # display("correct_affilations: ", correct_affilations)

    # -> Trovato profilo corretto !

    # Creo DataFrame
    author_id = correct_affilations.get('author_id')
    cited_by = correct_affilations.get('cited_by', 'value')
    interests = correct_affilations.get('interests')
    title = None
    for interest in interests: # interests dizionario -> estraggo title
        title = interest.get('title', '')

    # print(author_id)
    # print(cited_by)
    # print(title)

    # non funziona append!
    new_row = {'author_id': author_id, 'cited_by': cited_by, 'interests': title}
    df_nodes_aux.loc[len(df_nodes_aux)] = new_row

    # esco dal for

# aggiorno il file originale con apposite colonne
df = pd.concat([df_nodes, df_nodes_aux], axis=1)
# display(df)

# memorizzo dataframe
df.to_csv("data/nodes.csv", index=False)

In [9]:
# 2. Per ciascuno dei 7 autori, utilizzare il suo ID per accedere al relativo profilo Google
#    Scholar e scaricare l’elenco dei suoi coautori, sempre via SerpAPI. Con tale elenco di nomi:
# a. Utilizzare le SerpAPI per cercare su Google Scholar un ricercatore che corrisponde a tale nome. 
#    Per ciascuno, salvare name, affiliations, author_id, cited_by e interests in un nuovo DataFrame contenente tutte
#    queste informazioni relative ai coautori dei 7 autori originari.
# b. Concatenare il DataFrame con i 7 autori originari e quello dei coautori generato al punto 2a in un unico DataFrame.
# NOTA BENE: è sufficiente effettuare la ricerca dei profili per nome, non accedere al loro profilo tramite id.
# ASSUNZIONE: in questo caso non potete identificare il profilo corretto tramite il valore di affiliations, 
# quindi assumete che quello corretto sia il primo ritornato nella lista di authors.
# c. Creare un terzo DataFrame con le colonne author1, author2 che rappresenta le co-authorship. 
# In tale DataFrame, una riga rappresenta un arco di coauthorship tra due autori.

# Leggo file csv e carico dataframe
df_nodes = pd.read_csv("data/nodes.csv")

# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# Creo dataframe ausiliario per i coautori
#df_coauthor = pd.DataFrame(
#    columns=["coauthor_name", "coauthor_affiliations", "coauthor_author_id", "coauthor_cited_by",
#             "coauthor_interests"])
df_coauthor = pd.DataFrame(columns=["name", "affiliations", "author_id", "cited_by", "interests"])
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

df_edges = pd.DataFrame(columns=["author1", "author2"])
                            
for name, author_id in zip(df_nodes["name"], df_nodes["author_id"]):
        
    # print(author_id)
        
    params = {
        "engine": "google_scholar_author",
        "hl": "en",
        "author_id": author_id
    }

    search = GoogleScholarSearch(params)
    results_author_id = search.get_dict()  # elenco di coautori per ciascun autore

    # a. Utilizzare le SerpAPI per cercare su Google Scholar un ricercatore che
    # corrisponde a tale nome. Per ciascuno, salvare name, affiliations,
    # author_id, cited_by e interests in un nuovo DataFrame contenente tutte
    # queste informazioni relative ai coautori dei 7 autori originari.

    for coauthor in results_author_id["co_authors"]:

        coauthor_name = coauthor['name']

        # trova ricercatore che corrisponde a tale nome

        # Ricerco name tra i profili presenti su Google Scholar
        params = {
            "engine": "google_scholar_profiles",
            "hl": "en",
            "mauthors": coauthor_name
        }

        search = GoogleScholarSearch(params)

        # prendo primo profilo
        results_author_id_profile = search.get_dict()["profiles"][0]
        results_author_id_profile1 = results_author_id_profile
        # print(results_author_id_profile1)
            
        # Per ciascuno, salvare name, affiliations, author_id, cited_by e interests
        # in un nuovo DataFrame contenente tutte queste informazioni relative ai coautori
        # dei 7 autori originari.
              
        # Creo variabili:
        coauthor_affiliations = results_author_id_profile1.get('affiliations')
        # print(coauthor_affilations)
        coauthor_id = results_author_id_profile1.get('author_id')
        # print(coauthor_id)
        coauthor_cited_by = results_author_id_profile1.get('cited_by', 'value')
        # print(coauthor_cited_by)
        coauthor_interests = results_author_id_profile1.get('interests')
        # print(coauthor_interests)
        coauthor_title = None

        # title può essere None -> se vero lo imposto null
        try: 
            for interest in coauthor_interests:
                coauthor_title = interest.get('title', '')
        except:
            coauthor_title = ""

        # non funziona append!
        # creo riga da aggiungere a df_coauthor
        new_row_coauthor = {'name': coauthor_name, 'affiliations': coauthor_affiliations, 'author_id': coauthor_id, 'cited_by': coauthor_cited_by, 'interests': coauthor_title}
        
        # aggiungo riga a df_coauthor
        df_coauthor.loc[len(df_coauthor)] = new_row_coauthor

        #name, coauthor_name
            
        new_row_edges = {'author1': name, 'author2': coauthor_name}
            
        df_edges.loc[len(df_edges)] = new_row_edges

            
        #print(new_row)
            
        # esco dal secondo for
    # esco dal primo for
    
display(df_coauthor)
    
# b. Concatenare il DataFrame con i 7 autori originari e quello dei coautori
# generato al punto 2a in un unico DataFrame.
# NOTA BENE: è sufficiente effettuare la ricerca dei profili per nome, non
# accedere al loro profilo tramite id.
# ASSUNZIONE: in questo caso non potete identificare il profilo corretto tramite
# il valore di affiliations, quindi assumete che quello corretto sia il primo
# ritornato nella lista di authors.

# salvataggio non neccessario -> eseguito solo per velocizzare il programma
df_coauthor.to_csv("data/coauthor.csv", index=False)

# concateno il DataFrame con i 7 autori originari e quello dei coautori
df_nodes = pd.concat([df_nodes, df_coauthor], ignore_index=True)

# c. Creare un terzo DataFrame con le colonne author1, author2 che
# rappresenta le co-authorship. In tale DataFrame, una riga rappresenta un
# arco di coauthorship tra due autori.
# ESEMPIO: David La Barbera, Michael Soprano è una riga del DataFrame
# creato al punto 2c se Michael Soprano è coautore di David La Barbera. La
# co-authorship è binaria, non pesata.

    
# df_edges = pd.DataFrame(columns=["author1", "author2"])

# df_edges = pd.concat([df_nodes["name"], df_coauthor["coauthor_name"]], axis=1)

df_edges.to_csv("data/edges.csv", index=False)

# A questo punto avrete prodotto due DataFrame:
# - Uno per le informazioni relative agli autori (originali + i relativi coautori) e 
#   contenente per ciascuno di essi i valori di name, affiliations, cited_by, interests. 
#   Salvare come nodes.csv nella cartella /data tale DataFrame.
# - Uno per le relazioni di co-authorship dai 7 autori principali verso i relativi coautori con colonne author1, author2. 
#   Salvare come edges.csv nella cartella /data tale DataFrame

,name,affiliations,author_id,cited_by,interests
0,Kevin Roitero,University of Udine,1xd52jMAAAAJ,690,Information Retrieval
1,Stefano Mizzaro,Full professor of Computer Science and Informa...,2wvJC6IAAAAJ,4739,misinformation and fake news detection
2,Gianluca Demartini,Associate Professor at the University of Queen...,PCAiILsAAAAJ,5348,Crowdsourcing
3,Damiano Spina,"School of Computing Technologies, RMIT University",sLzYrNYAAAAJ,2483,Human-AI Fact-checking
4,Michael Soprano,Postdoctoral Research Fellow at the University...,ocK0qRUAAAAJ,210,Crowdsourcing
...,...,...,...,...,...
122,Pavlos Vougiouklis,Huawei Technologies,9J7YeR0AAAAJ,576,
123,Jonathon Hare,"NOAA Fisheries, Northeast Fisheries Science Ce...",d_HvmVsAAAAJ,12279,management
124,Carlo Alberto Beltrami,"Professore, Università di Udine",EZrMWWMAAAAJ,18388,cellule staminali
125,Richard Gomer,"Professor of Biology, Texas A&M University",z1YBrC4AAAAJ,10316,cell density sensing
